In [12]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time

class System: # The array within which the molecules will sit
    
    def __init__(self, mol_len, num_mols, atom_sequence = 0, density = 1): # Default density is 1 gram per cm^3
        self.mol_len = mol_len
        self.num_mols = num_mols
        self.atom_sequence = atom_sequence
        self.density = density
        self.all_paths = []
        self.sys_hist = []
        self.array = np.array([0,0])

    def save(self, array, sys_state, ):
        """Saves the system such that it can be rolled back."""
        sys_state = np.copy(array)
        self.sys_hist.append(sys_state)
        
        return sys_state

    def undo(self):
        """Returns system state back to before the random moves were made."""
        self.array = sys_state
        return array

    def mega_undo(self, array, sys_state):
        """Reverts system state back to a previous version."""
        self.sys_state = self.sys_hist[-1]
        self.array = self.sys_state
        self.sys_hist = self.sys_hist[:-1]
    
    def walk(self):
        """Creates a path taken by a Monte-Carlo simulation to make a chain of atoms."""
        box_length = int(np.ceil(np.sqrt(self.mol_len / self.density))) # Calculates requisite box size
        self.array = self.array.resize((box_length, box_length)) # The 'box'
        moves = np.array([[1,0],[-1,0],[0,1],[0,-1],[1,1],[-1,1],[1,-1],[-1,-1]]) # Possible moves for the random walk
        
        for mol in self.num_mols:
            path = []
            position = np.array([random.randint(0, box_length - 1), random.randint(0, box_length - 1)])
            self.array[position[0], position[1]] = 1
            path.append(position.tolist())
            i = 0 # Count of atoms placed
            j = 0 # Count of rejected moves
            move = moves[random.randint(0,7)]
            
    

PPS = System(100,2)
PPS.walk()

NameError: name 'num_mols' is not defined